<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Carolina Nuñez
- Nombre de alumno 2: Alonso Uribe


### **Link de repositorio de GitHub:** [Repositorio💻](https://github.com/carinunez/Labs_MDS/)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

## 1 Generando un Baseline (5 puntos)

<!-- <p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p> -->

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
from sklearn import set_config
set_config(transform_output="pandas")

from sklearn.model_selection import train_test_split
from  sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [ ]:
X = df.drop(columns='quantity')
y = df.quantity
X_train_val, X_test, y_train, y_test= train_test_split(X, y,  train_size=0.9, test_size=0.1, random_state=29)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train, train_size=0.7, test_size=0.2, random_state=29)

In [ ]:
def extract_from_date(df):
  # Le entrego una pd.series a la funcion
  df['day'] = df.date.apply(lambda x: x.split('/')[0]).astype('category')
  df['month'] = df.date.apply(lambda x: x.split('/')[1]).astype('category')
  df['year'] = df.date. apply(lambda x: x.split('/')[2]).astype('category')
  return df

fxn_transf = FunctionTransformer(extract_from_date)
# df = fxn_transf.fit_transform(df)

In [ ]:
numerical_cols = (df.drop(columns=['id', 'quantity']).select_dtypes('number').columns.to_list())
cat_cols = (df.drop(columns=['id', 'quantity', 'date']).select_dtypes(exclude='number').columns.to_list()) +['day', 'month', 'year']
# print('Num cols:', numerical_cols)
# print('Cat cols:',cat_cols)

In [ ]:
one_hot = OneHotEncoder(sparse_output=False)
col_tr = ColumnTransformer([
                            # ('mantener_id','passthrough', ['id']),
                            ('drop_date','drop', ['id','date']),
                            ('num_cols', MinMaxScaler(), numerical_cols),
                            ('cat_cols', one_hot, cat_cols)],
                           remainder="passthrough",
                           verbose_feature_names_out=False)
col_tr.set_output(transform='pandas')

In [ ]:
pipe_dummy = Pipeline([
                ('extract_date', FunctionTransformer(extract_from_date)),
                ('col_transformation', col_tr),
                ('dummy', DummyRegressor(strategy='mean'))])
pipe_dummy.fit(X_train, y_train)
y_val_pred = pipe_dummy.predict(X_val)
print('MAE :', mean_absolute_error(y_val, y_val_pred))

El MAE obtenido es alto, por lo que se podría estar sobreestimando considerablemente la demanda.

In [ ]:
pipe_xgb = Pipeline([
                ('extract_date', FunctionTransformer(extract_from_date)),
                ('col_transformation', col_tr),
                ('xgb_reg', XGBRegressor(enable_categorical=True))])
pipe_xgb.fit(X_train, y_train)
y_val_pred = pipe_xgb.predict(X_val)
print('MAE :', mean_absolute_error(y_val, y_val_pred))

El MAE obtenido usando XGBRegressor es considerablemente menor por lo que este permitiría predecir de mejor manera la demanda que al utilizar DummyRegressor.

In [ ]:
import pickle

dummy_pkl = "dummy_model.pkl"
xgb_pkl = "xgb_model.pkl"

with open(dummy_pkl, 'wb') as file:
    pickle.dump(pipe_dummy, file)

with open(xgb_pkl, 'wb') as file:
    pickle.dump(pipe_xgb, file)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
pipe_xgb_neg = Pipeline([
                ('extract_date', FunctionTransformer(extract_from_date)),
                ('col_tr', col_tr),
                ('xgb_reg', XGBRegressor(enable_categorical=True,
                                         monotone_constraints={'price':-1}
                                         ))
                ])

# pipe_xgb_neg.set_params(xgb_reg__monotone_constraints={'price':-1})
pipe_xgb_neg.fit(X_train, y_train)
y_val_pred = pipe_xgb_neg.predict(X_val)
print('MAE :', mean_absolute_error(y_val, y_val_pred))

xgb_neg_pkl = "xgb_neg_model.pkl"
with open(xgb_neg_pkl, 'wb') as file:
    pickle.dump(pipe_xgb_neg, file)

In [ ]:
# pipe_xgb_neg.get_params()

El error aumentó por lo que incluir esta restricción, no sería buena idea.

In [ ]:
feature_names = pipe_xgb_neg.named_steps['col_tr'].get_feature_names_out()
# print(len(feature_names))

Al incluir esta relación, se logra disminuir ligeramente el MAE

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)


Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    trial.set_user_attr("best_model", 128)
    monotone_constraints_str = "(" + ",".join(["-1" if col == 'price' else "0" for col in X_val.columns]) + ")"

    pipe_xgb_neg.set_params(
        col_transformation__cat_cols__min_frequency=trial.suggest_float("min_frequency", 0.0, 1.0),
        xgb_reg__learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1),
        xgb_reg__n_estimators = trial.suggest_int("n_estimators", 50, 1000),
        xgb_reg__max_depth = trial.suggest_int("max_depth", 3, 10),
        xgb_reg__max_leaves = trial.suggest_int("max_leaves", 0, 100),
        xgb_reg__min_child_weight = trial.suggest_int("min_child_weight", 1, 5),
        xgb_reg__reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0),
        xgb_reg__reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0),
        xgb_reg__monotone_constraints=monotone_constraints_str
        )

    # Predict and evaluate the model
    y_pred = pipe_xgb_neg.predict(X_val)
    return mean_absolute_error(y_val, y_pred)

In [ ]:
study = optuna.create_study(sampler=TPESampler(), direction="minimize")

In [ ]:
study.optimize(objective, n_trials=50)

In [ ]:
print('Menor MAE: ', study.best_value)
print('Mejores params: ', study.best_params)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>